In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import timeit
import numba
from numba import jit

from iteration_methods import*
from basic import*

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [4]:
@jit(nopython=True)
def sor_iteration(matrix, source, grid, init_guess=None, boundary=((0, 0), (0,0)), tolerance=1.e-8, itermax=1000, omega=1.5):

    M, f, u0, B, tol, kmax, w = matrix, source, init_guess, boundary, tolerance, itermax, omega

    x, y = grid

    Nx, Ny = x.shape[1]-1, y.shape[0]-1

    dx, dy = (x[0, -1] - x[0, 0])/Nx, (y[-1, 0] - y[0, 0])/Ny

    if init_guess is None:

        u0 = np.ones_like(f)

    #assign dirichlet boundary conditions
    u0[:, 0] = B[0][0]                               
    u0[:, -1] = B[0][1]
    u0[0, :] = B[1][0]
    u0[-1, :] = B[1][1]
    
    u = u0
    k = 0
    rel_diff = tol + 1
    conv_hist = []
       
    #iteration loop using nested loops.
    while  k < kmax and rel_diff > tol:

        u_next = u.copy()

        for j in range(1, Ny):
            for i in range(1, Nx):
                
                u_next[j, i] = (1-omega)*u[j,i] + omega*((u_next[j, i-1] + u[j, i+1])*dy**2 + 
                                (u_next[j-1, i] + u[j+1, i])*dx**2 - f[j,i]*dx**2*dy**2)/(2*(dx**2+dy**2))
        
        

        rel_diff = la.norm(u_next-u)/la.norm(u)
        conv_hist.append(rel_diff)
        u = u_next
        k += 1

    return u, k, rel_diff, conv_hist

In [5]:
Nx = 50
Ny = 20

x_I, x_F = 0, 1
y_I, y_F = 0, 1

X = np.linspace(x_I, x_F, Nx+1)
Y = np.linspace(y_I, y_F, Ny+1)

dx = (x_F-x_I)/(Nx+1)
dy = (y_F-y_I)/(Ny+1)

x, y = np.meshgrid(X, Y)

In [6]:
#Set up the solver parameters

X, Y, x, y, dx, dy = grid()                             #The grid of the problem

f = source(x, y)                                        #source term

u_guess = np.sin(np.pi*x/2)*np.cos(np.pi*y)             #initial guess

itermax = 20000                                         #maximum number of iterations

tol = 1e-8                                              #desired tolerance

omega = 1.5  #2/(1+np.sin(np.pi/Nx))

boundary = ((0,1), (X, X))                              #dirichlet boundary conditions
# print(omega_opt)

In [7]:
#Test the solver
omega_opt_x = 2/(1+np.sin(np.pi*dx))
omega_opt_y = 2/(1+np.sin(np.pi*dy))

omega=np.linspace(1.8, 2, 20)
omega = [1.5, 1.89, omega_opt_x, omega_opt_y, 1]

l_iternum = []
l_elapsed = []
l_rel_diff = []
l_error = []
l_u = []

for i in range(len(omega)-1):
    
    #calculate the solution    
    start = timeit.default_timer()            #time the solver

    u, iternum, rel_diff, conv_hist = sor_iteration(np.eye(Nx), f, (x, y), init_guess=u_guess, 
                                                    boundary=boundary, tolerance=tol, itermax=itermax, omega=omega[i])       

    end = timeit.default_timer()

    elapsed = (end - start)

    u_an = u_analytical(x, y)                #analytical solution for comparison

    error = la.norm(u - u_an, 2)/la.norm(u_an, 2)

    #Keep results in lists for processing
    l_iternum.append(iternum)
    l_rel_diff.append(rel_diff)
    l_elapsed.append(elapsed)
    l_error.append(error)
    l_u.append(u)
    if iternum == itermax:

        print('WARNING: desired tolerance has not been reached for the given maximum iterations \n')

    print('Omega = {:1.4f} \n'
        'Number of iterations: {} \n'
        'Last relative difference: {:1.4e} \n'
        'Error to analytical: {:1.4E} \n'
        'Time elapsed: {:1.2e} s'.format(omega[i], iternum, rel_diff, error, elapsed))
    print('-'*50)

    

Omega = 1.5000 
Number of iterations: 705 
Last relative difference: 9.8575e-09 
Error to analytical: 5.9420E-04 
Time elapsed: 1.95e+00 s
--------------------------------------------------
Omega = 1.8900 
Number of iterations: 171 
Last relative difference: 8.6538e-09 
Error to analytical: 5.9466E-04 
Time elapsed: 2.90e-03 s
--------------------------------------------------
Omega = 1.8840 
Number of iterations: 157 
Last relative difference: 9.8780e-09 
Error to analytical: 5.9468E-04 
Time elapsed: 3.10e-03 s
--------------------------------------------------
Omega = 1.7406 
Number of iterations: 318 
Last relative difference: 9.8123e-09 
Error to analytical: 5.9448E-04 
Time elapsed: 4.84e-03 s
--------------------------------------------------


In [8]:
fig, (ax1, ax2) = plt.subplots(2)

ax1.plot(omega[:-1], np.array(l_elapsed))
ax1.set_xlabel('omega')
ax1.set_ylabel('elapsed time')

ax2.plot(omega[:-1], np.array(l_iternum), c='r')
ax2.set_xlabel('omega')
ax2.set_ylabel('number of iterations')

Text(0, 0.5, 'number of iterations')

In [9]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


scat = ax.scatter(x, y, l_u[-1], c = 'r', label = 'numerical', alpha = 1)
surf = ax.plot_surface(x, y, u_an, cmap = cm.coolwarm, label = 'analytical', alpha = 0.3)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('$u = \sin\pi x \sin\pi y + x$')
# ax.set_zlim(top=2., bottom=-0.5)

surf._facecolors2d=surf._facecolors3d
surf._edgecolors2d=surf._edgecolors3d
ax.legend(loc = 2)
plt.show()